# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom rule-based agent. 

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/ 

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev` 
4. Install gymnasium with the needed extras

In [17]:
%pip install gymnasium[box2d,classic_control]


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## The Lunar Lander Environment 

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use 
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Define an environment function based on the example in the documentation.

In [18]:
import gymnasium as gym

def run_episode(agent):
    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent function to select an action
        action = agent(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)
    
        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break
    
    env.close()
    return reward

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [19]:
import numpy as np


# define a random agent
def random_agent(observation): 
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent)

Obs: [-0.00440578  1.4097735  -0.44627643 -0.0509764   0.00511202  0.1010883
  0.          0.        ] -> Action: 0
Obs: [-0.00881195  1.4080495  -0.44567275 -0.07664685  0.0101072   0.0999134
  0.          0.        ] -> Action: 1
Obs: [-0.01329842  1.40572    -0.4557354  -0.10359526  0.01711529  0.14017501
  0.          0.        ] -> Action: 1
Obs: [-0.0178731   1.4027802  -0.4667944  -0.13079979  0.02633836  0.18447825
  0.          0.        ] -> Action: 0
Obs: [-0.02244816  1.3992413  -0.46682248 -0.15747365  0.0355594   0.18443784
  0.          0.        ] -> Action: 1
Obs: [-0.02712011  1.3950906  -0.4789761  -0.18479748  0.04721835  0.2332007
  0.          0.        ] -> Action: 0
Obs: [-0.03179245  1.3903418  -0.47901115 -0.21147546  0.05887433  0.23314095
  0.          0.        ] -> Action: 0
Obs: [-0.03646536  1.3849946  -0.4790454  -0.238154    0.07052889  0.2331126
  0.          0.        ] -> Action: 1
Obs: [-0.04123039  1.3790383  -0.4905971  -0.26544794  0.08450158  0

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [20]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [21]:
def rocket_agent(observation):
    """A simple agent."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.1:  
        return Act.MAIN.value

    return Act.NO_OP.value 

run_episode(rocket_agent)

Obs: [-0.00464048  1.4000013  -0.4700353  -0.4852881   0.00538385  0.10646997
  0.          0.        ] -> Action: 2
Obs: [-0.00932846  1.389096   -0.47390023 -0.48470306  0.01042775  0.10088842
  0.          0.        ] -> Action: 2
Obs: [-0.01388655  1.3782911  -0.46156254 -0.480275    0.01611682  0.11379214
  0.          0.        ] -> Action: 2
Obs: [-0.01847515  1.3680022  -0.46452865 -0.4573592   0.02171599  0.11199403
  0.          0.        ] -> Action: 2
Obs: [-0.02308483  1.3577418  -0.46656838 -0.45610243  0.02725842  0.11085857
  0.          0.        ] -> Action: 2
Obs: [-0.02778263  1.3479074  -0.47504106 -0.43719262  0.03246862  0.10421345
  0.          0.        ] -> Action: 2
Obs: [-0.03246298  1.3381057  -0.4734506  -0.43576157  0.0378271   0.10717962
  0.          0.        ] -> Action: 2
Obs: [-0.03721295  1.3283638  -0.48015994 -0.43311626  0.04293271  0.10212147
  0.          0.        ] -> Action: 2
Obs: [-0.04195805  1.3194902  -0.47989973 -0.39453948  0.0482608

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [22]:
import numpy as np

def run_episode_test(agent):
    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)
      
        if terminated:
            break
    
    env.close()
    return reward

def run_episodes(agent, n=100):
    return [run_episode_test(agent) for _ in range(n)]


rewards = run_episodes(rocket_agent)
print(rewards)

print(f"Average reward: {np.average(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Average reward: -100.0


This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [23]:
# Code goes here